In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [17]:
import glob
import numpy as np
import pandas as pd

from Bio import Phylo, SeqIO

In [3]:
import rooting_methods
import rooting_methods_weighted

**Toggle below to analyze different datasets**

In [165]:
# input_dir = '../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/'
# output_dir = '../Data/Tria_et_al_data/eukaryotes/processed_trees/'
input_dir = '../Data/OMA_group_data/eukaryotes/raw_OMA_trees/'
output_dir = '../Data/OMA_group_data/eukaryotes/processed_OMA_trees/'

# Mid-point rooting first
(Also labeling all internal nodes)

In [166]:
import random
def trim_zero_bls(my_tree):
    zero_bls = [term for term in my_tree.get_terminals() if term.branch_length==0.]
    while len(zero_bls)>0:
        my_tree.prune(random.choice(zero_bls))
        zero_bls = [term for term in my_tree.get_terminals() if term.branch_length==0.]
    return my_tree

In [154]:
for input_tree in glob.glob(input_dir+'*')[0:200]:#SUBSET?
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=False)
    for i,j in enumerate(my_tree.get_nonterminals()):
        j.name = 'IntNode_{}'.format(i)
        j.confidence = None
    my_tree = rooting_methods.mp_root_adhock(my_tree)
    assert my_tree.is_bifurcating()
    outfile_name = input_tree.split('/')[-1].replace('.mafft.afa', '')
    Phylo.write(my_tree, '{}{}.Rooted.MPAJH'.format(output_dir, outfile_name), format='newick')
    ###Repeat but get rid of zero length terminals
    my_tree = trim_zero_bls(my_tree)
    my_tree = rooting_methods.mp_root_adhock(my_tree)
    assert my_tree.is_bifurcating()
    Phylo.write(my_tree, '{}{}.NoZeroTerms.Rooted.MPAJH'.format(output_dir, outfile_name), format='newick')

../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0436.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0166.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0810.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG2407.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG3745.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG2157.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG2035.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG1879.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0554.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0446.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG1704.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0116.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG3265.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes/ingroup/phyml/KOG0860.faa.aln.nwk
../Data/Tria_et_al_data/eukaryotes

# Minimizing root-to-tip deviations

In [155]:
for input_tree in glob.glob(output_dir+'*.Rooted.MPAJH')[:]:
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=True)
    starting_bl = my_tree.total_branch_length()
    new_tree, function_optima, depths_dict = rooting_methods.ml_root_adhock(my_tree)
    ending_bl = new_tree.total_branch_length()
    assert new_tree.is_bifurcating()
    assert np.isclose(starting_bl, ending_bl)
    outfile_name = input_tree.replace('.MPAJH', '.MINSDAJH')
    Phylo.write(new_tree, outfile_name, format='newick')

../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0822.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4067.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3181.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG1248.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3980.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0743.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3049.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0385.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4077.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0495.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4017.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/euka

# My MAD implementation

In [156]:
for input_tree in glob.glob(output_dir+'*.Rooted.MPAJH')[:]:
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=True)
    starting_bl = my_tree.total_branch_length()
    new_tree, RAI, function_optima = rooting_methods.mad_root_adhock(my_tree)
    ending_bl = new_tree.total_branch_length()
    assert new_tree.is_bifurcating()
    assert np.isclose(starting_bl, ending_bl)
    outfile_name = input_tree.replace('.MPAJH', '.MADAJH')
    Phylo.write(new_tree, outfile_name, format='newick')

../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0822.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4067.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3181.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG1248.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3980.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0743.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3049.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0385.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4077.faa.aln.nwk.NoZeroTerms.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0495.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4017.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/euka

# My (very preliminary) weighted methods

In [16]:
# for input_tree in glob.glob('../Data/raw_OMA_trees/*.Rooted.MPAJH')[:]:
# # for input_tree in glob.glob('../Data/euk_trees/*.Rooted.MPAJH')[:]:
#     if os.path.exists(input_tree.replace('.MPAJH', '.MLWEIGHTAJH')):
#         continue
#     print(input_tree)
#     my_tree = Phylo.read(input_tree, 'newick', rooted=True)
#     print(my_tree.total_branch_length())
#     new_tree, function_optima, depths_dict, weights_dict = rooting_methods_weighted.ml_root_weighted(my_tree)
#     print(new_tree.total_branch_length())
#     assert new_tree.is_bifurcating()
#     assert np.isclose(new_tree.total_branch_length(), my_tree.total_branch_length())
#     outfile_name = input_tree.replace('.MPAJH', '.MLWEIGHTAJH')
#     Phylo.write(new_tree, outfile_name, format='newick')

# Assessing monophyly of each tree for testing purposes for the OMA set of data (skip this entire section if running for a different dataset)
And grabbing the internal node names where the monophyly exists

In [167]:
def is_monophyletic_all_fast(hypothetical_root, previous_root, tree, test_set, monophyletic_list):
    if hypothetical_root == tree.root:
        l_clade = hypothetical_root.clades[0]
        r_clade = hypothetical_root.clades[1]
        l_terms = set(l_clade.get_terminals())
        r_terms = set(r_clade.get_terminals())
        if len(test_set.union(l_terms))-len(test_set) == 0 and\
            len(test_set.intersection(r_terms)) == 0:
            monophyletic_list.append((l_clade.name, r_clade.name, len(l_terms)))
        elif len(test_set.union(r_terms))-len(test_set) == 0 and\
            len(test_set.intersection(l_terms)) == 0:
            monophyletic_list.append((r_clade.name, l_clade.name, len(r_terms)))
    else:
        ds_terms = set(hypothetical_root.get_terminals())
        us_terms = set(tree.get_terminals()) - ds_terms
        if len(test_set.union(ds_terms))-len(test_set) == 0 and\
            len(test_set.intersection(us_terms)) == 0:
            monophyletic_list.append((hypothetical_root.name, previous_root.name, len(ds_terms)))
        if len(test_set.union(us_terms))-len(test_set) == 0 and\
            len(test_set.intersection(ds_terms)) == 0:
            monophyletic_list.append((previous_root.name, hypothetical_root.name, len(us_terms)))
    if len(hypothetical_root.clades) == 2:
        l_clade, r_clade = hypothetical_root.clades
        monophyletic_list = is_monophyletic_all_fast(l_clade, hypothetical_root, tree, test_set, monophyletic_list)
        monophyletic_list = is_monophyletic_all_fast(r_clade, hypothetical_root, tree, test_set, monophyletic_list)
    elif len(hypothetical_root.clades) == 0:
        return monophyletic_list
    return monophyletic_list

def get_subsets(oma_group):
    seqs = SeqIO.parse('../Data/OMA_group_data/eukaryotes/raw_OMA_groups/Dikarya_OMA/{}.fa'.format(oma_group), 'fasta')
    species = []
    for seq in seqs:
        species.append(seq.id)
    return species

trees_dir = '../Data/OMA_group_data/eukaryotes/processed_OMA_trees/*treefile.Rooted.MPAJH'
output_csv_file = '../Data/OMA_group_data/eukaryotes/tree_monophyly.csv'
monophyly_df = pd.DataFrame(columns=['Group_id', 'monophyletic_clade', 'other_clade'])
for tree_loc in glob.glob(trees_dir)[:]:
    print(tree_loc)
    oma_group = tree_loc.split('/')[-1].split('.')[0]
    test_tree = Phylo.read(tree_loc, 'newick', rooted=True)
    subset_names = get_subsets(oma_group)
    testy = [term for term in test_tree.get_terminals() if\
                      term.name in subset_names]
    test_set = set(testy)
    possible_clades = is_monophyletic_all_fast(test_tree.root, False, test_tree, test_set, [])
    if len(possible_clades) >= 1:
        clades_of_interest = list(set(possible_clades))
        clades_of_interest = [(a,b,c) for a,b,c in clades_of_interest if None not in (a,b,c)]
        assert len(clades_of_interest) == 1
        clade_of_interest = clades_of_interest[0]
    else:
        clade_of_interest = ('', '', '')
    monophyly_df = monophyly_df.append({'Group_id':oma_group,
                                    'monophyletic_clade':clade_of_interest[0], 
                                    'other_clade':clade_of_interest[1]}, ignore_index=True)    

../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_733878.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_754919.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_788814.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_833097.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_803985.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_555520.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_819101.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_786887.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_761172.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_555739.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_830349.treefile.R

In [168]:
monophyly_df.head()

,Group_id,monophyletic_clade,other_clade
0,OMAGroup_733878,IntNode_66,IntNode_91
1,OMAGroup_754919,IntNode_25,IntNode_90
2,OMAGroup_788814,IntNode_45,IntNode_53
3,OMAGroup_833097,IntNode_1,IntNode_90
4,OMAGroup_803985,IntNode_41,IntNode_53


In [169]:
monophyly_df.to_csv(output_csv_file, index=False)

# Assessing monophyly for Tria *et al.* dataset

In [157]:
trees_dir = '../Data/Tria_et_al_data/eukaryotes/processed_trees/*.nwk.Rooted.MPAJH'
output_csv_file = '../Data/Tria_et_al_data/eukaryotes/tree_monophyly.csv'
monophyly_df = pd.DataFrame(columns=['Group_id', 'monophyletic_clade', 'other_clade'])

subset_names = []
with open('../Data/Tria_et_al_data/eukaryotes/ID_to_Species.txt', 'r') as infile:
    texty = infile.readlines()
    for line in texty[1:]:
        sl = line.split('\t')
        if sl[2].strip() == 'f':
            subset_names.append(sl[0].strip())
            
for tree_loc in glob.glob(trees_dir)[:]:
    print(tree_loc)
    group_id = tree_loc.split('/')[-1].split('.')[0]
    test_tree = Phylo.read(tree_loc, 'newick', rooted=True)
    testy = [term for term in test_tree.get_terminals() if\
                      term.name in subset_names]
    test_set = set(testy)
    possible_clades = is_monophyletic_all_fast(test_tree.root, False, test_tree, test_set, [])
    if len(possible_clades) >= 1:
        clades_of_interest = list(set(possible_clades))
        clades_of_interest = [(a,b,c) for a,b,c in clades_of_interest if None not in (a,b,c)]
        assert len(clades_of_interest) == 1
        clade_of_interest = clades_of_interest[0]
    else:
        clade_of_interest = ('', '', '')
    monophyly_df = monophyly_df.append({'Group_id':group_id,
                                    'monophyletic_clade':clade_of_interest[0], 
                                    'other_clade':clade_of_interest[1]}, ignore_index=True)  

../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0822.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG1248.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3980.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0743.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG3049.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0495.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4017.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG2971.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG0356.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG4093.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG2157.faa.aln.nwk.Rooted.MPAJH
../Data/Tria_et_al_data/eukaryotes/processed_trees/KOG1907.faa.aln.nwk.Roote

In [158]:
monophyly_df.head()

,Group_id,monophyletic_clade,other_clade
0,KOG0822,IntNode_12,IntNode_16
1,KOG1248,IntNode_13,IntNode_12
2,KOG3980,,
3,KOG0743,IntNode_12,IntNode_16
4,KOG3049,IntNode_5,IntNode_9


In [159]:
monophyly_df.to_csv(output_csv_file, index=False)

# Add useful stuff to the monophyly dataframe

**Toggle below for different input data types**

In [170]:
input_csv_file = '../Data/OMA_group_data/eukaryotes/tree_monophyly.csv'
trees_dir = '../Data/OMA_group_data/eukaryotes/processed_OMA_trees/*.treefile.Rooted.MPAJH'
# input_csv_file = '../Data/Tria_et_al_data/eukaryotes/tree_monophyly.csv'
# trees_dir = '../Data/Tria_et_al_data/eukaryotes/processed_trees/*.nwk.Rooted.MADAJH'
#
#
#
monophyly_df = pd.read_csv(input_csv_file, index_col='Group_id')

In [171]:
monophyly_df.head()
for tree_loc in glob.glob(trees_dir)[:]:
    print(tree_loc)
    file_name = tree_loc.split('/')[-1]
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=10e-10)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    monophyly_df.set_value(group, 'root_bl',
                           starting_tree.root.clades[1].branch_length+starting_tree.root.clades[0].branch_length)
    monophyly_df.set_value(group, 'monophyletic_total_bl', 
                           starting_tree.root.clades[1].total_branch_length() 
                           - starting_tree.root.clades[1].branch_length)
    monophyly_df.set_value(group, 'other_total_bl', 
                           starting_tree.root.clades[0].total_branch_length() 
                           - starting_tree.root.clades[0].branch_length)
    monophyly_df.set_value(group, 'total_tree_bl', 
                           starting_tree.total_branch_length())
    
    monophyly_df.set_value(group, 'total_n', len(starting_tree.get_terminals()))
    monophyly_df.set_value(group, 'monophyletic_n', len(starting_tree.root.clades[1].get_terminals()))
    monophyly_df.set_value(group, 'other_n', len(starting_tree.root.clades[0].get_terminals()))

../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_733878.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_754919.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_788814.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_833097.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_803985.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_555520.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_819101.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_786887.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_761172.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_555739.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/processed_OMA_trees/OMAGroup_830349.treefile.R

In [172]:
monophyly_df.head()

,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
OMAGroup_733878,IntNode_66,IntNode_91,0.52861,29.78150,55.18236,85.49247,203.0,92.0,111.0
OMAGroup_754919,IntNode_25,IntNode_90,0.12304,3.82319,1.20663,5.15286,204.0,92.0,112.0
OMAGroup_788814,IntNode_45,IntNode_53,1.15706,47.97359,29.46055,78.59120,200.0,78.0,122.0
OMAGroup_833097,IntNode_1,IntNode_90,0.19569,20.50439,27.28626,47.98634,214.0,91.0,123.0
OMAGroup_803985,IntNode_41,IntNode_53,2.17933,25.95126,32.27627,60.40686,165.0,54.0,111.0


In [173]:
###Testing
max(monophyly_df['root_bl'] + monophyly_df['monophyletic_total_bl'] +\
monophyly_df['other_total_bl'] - monophyly_df['total_tree_bl'])

7.1054273576010019e-14

In [174]:
monophyly_df.to_csv(input_csv_file)

**OFFLINE: make some pruned trees!**

.

.

.

.

.

**Back online**

# Dealing with the pruned tree sets

In [175]:
input_dir = '../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/'

In [181]:
for input_tree in glob.glob(input_dir+'*.treefile')[:]:
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=False)
    my_tree = rooting_methods.mp_root_adhock(my_tree)
    assert my_tree.is_bifurcating()
    outfile_name = input_tree + '.Rooted.MPAJH'
    Phylo.write(my_tree, outfile_name, format='newick')

../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_733878.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_555739.frac0.95.treefile
Potential for multiple midpoints. Choosing one at random
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_555520.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_790712.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_819101.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_665231.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_659254.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_802797.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_732528.frac0.95.treefile
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_788814.frac0.95.treefile
Potential for multiple midpoints. Choosing one at random
Potential for multiple midpoints. Ch

In [183]:
# for input_tree in glob.glob('../Data/pruned_euk_trees/*_15_notmeta.nwk.Rooted.MPAJH')[:]:
for input_tree in glob.glob(input_dir+'*.MPAJH')[:]:
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=True)
    new_tree, function_optima, depths_dict = rooting_methods.ml_root_adhock(my_tree)
    assert new_tree.is_bifurcating()
    assert np.isclose(new_tree.total_branch_length(), my_tree.total_branch_length())
    outfile_name = input_tree.replace('.MPAJH', '.MINSDAJH')
    Phylo.write(new_tree, outfile_name, format='newick')

../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_788814.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_555739.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_833097.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_803985.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_786887.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_732528.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_840944.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_819101.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_733878.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_754919.frac0.95.treefile.Rooted.MPAJH
../Data/OMA_group_da

In [185]:
# for input_tree in glob.glob('../Data/pruned_euk_trees/*_15_notmeta.nwk.Rooted.MPAJH')[:]:
for input_tree in glob.glob(input_dir+'*0.95*.MPAJH')[:]:
    print(input_tree)
    my_tree = Phylo.read(input_tree, 'newick', rooted=True)
    print(my_tree.total_branch_length())
    new_tree, RAI, function_optima = rooting_methods.mad_root_adhock(my_tree)
    print(new_tree.total_branch_length())
    assert new_tree.is_bifurcating()
    assert np.isclose(new_tree.total_branch_length(), my_tree.total_branch_length())
    outfile_name = input_tree.replace('.MPAJH', '.MADAJH')
    Phylo.write(new_tree, outfile_name, format='newick')

../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_788814.frac0.95.treefile.Rooted.MPAJH
55.02192000000001
55.02192
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_555739.frac0.95.treefile.Rooted.MPAJH
32.52446999999998
32.52447
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_833097.frac0.95.treefile.Rooted.MPAJH
26.048369999999988
26.04837
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_803985.frac0.95.treefile.Rooted.MPAJH
32.497910000000005
32.49791
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_786887.frac0.95.treefile.Rooted.MPAJH
35.98077999999998
35.98078
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_732528.frac0.95.treefile.Rooted.MPAJH
49.08051999999999
49.08052
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_840944.frac0.95.treefile.Rooted.MPAJH
23.062290000000008
23.06229
../Data/OMA_group_data/eukaryotes/pruned_OMA_trees/OMAGroup_819101.frac0.95.treefile.Rooted.MPAJH
47.43195999999999
47.431

In [ ]:
# for input_tree in glob.glob('../Data/pruned_OMA_trees/*.pruned_0.9_dikarya.treefile.Rooted.MPAJH')[:]:
#     print(input_tree)
#     my_tree = Phylo.read(input_tree, 'newick', rooted=True)
#     print(my_tree.total_branch_length())
#     new_tree, function_optima, depths_dict, weights_dict = rooting_methods_weighted.ml_root_weighted(my_tree)
#     print(new_tree.total_branch_length())
#     assert new_tree.is_bifurcating()
#     assert np.isclose(new_tree.total_branch_length(), my_tree.total_branch_length())
#     outfile_name = input_tree.replace('.MPAJH', '.MLWEIGHTAJH')
#     Phylo.write(new_tree, outfile_name, format='newick')